## Part 4: Fraud

In [1]:
%run "../scripts/ETL.py"

22/10/04 20:43:15 WARN Utils: Your hostname, Aishwaryas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.6.81 instead (on interface en0)
22/10/04 20:43:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/04 20:43:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


AnalysisException: Cannot resolve column name "postcodes" among (id, postcode, locality, state, long, lat, dc, type, status, sa3, sa3name, sa4, sa4name, region, Lat_precise, Long_precise, SA1_MAINCODE_2011, SA1_MAINCODE_2016, SA2_MAINCODE_2016, SA2_NAME_2016, SA3_CODE_2016, SA3_NAME_2016, SA4_CODE_2016, SA4_NAME_2016, RA_2011, RA_2016, MMM_2015, MMM_2019, ced, altitude, chargezone, phn_code, phn_name, lgaregion, electorate, electoraterating)

The consumer fraud data shows the list of transactions with a high probability of being fraudulent, and therefore should be considered invalid.

In [2]:
print(fraud_consumer.count())
fraud_consumer = fraud_consumer.withColumnRenamed("fraud_probability","cons_fraud_prob").withColumnRenamed("order_datetime","order_datetime_1").withColumnRenamed("user_id","user_id_1")
fraud_consumer.limit(5)

34864


user_id_1,order_datetime_1,cons_fraud_prob
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508035


Probability of fraud for high revenue merchants...?

In [3]:
print(fraud_merchants.count())
fraud_merchants = fraud_merchants.withColumnRenamed("merchant_abn","merchant_abn_1").withColumnRenamed("fraud_probability","merch_fraud_prob")
fraud_merchants.limit(5)

114


merchant_abn_1,order_datetime,merch_fraud_prob
19492220327,2021-11-28,44.403658647495355
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


In [4]:

merchant_data = final_join.select("merchant_abn", "categories","revenue_levels","dollar_value","user_id","consumer_id")

- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [5]:
#Joining consumer fraud, merchant data, and merchant fraud data to one datatset: 

#Joining consumer fraud and merchant data together

merchant_data.createOrReplaceTempView("temp")

fraud_consumer.createOrReplaceTempView("temp2")

join_1 = spark.sql(""" 

SELECT *
FROM temp

INNER JOIN temp2

ON temp.user_id = temp2.user_id_1 
""")


In [6]:
#Joining join_1 to merchant fraud 

#Joining all test + predicting Sets Together 
join_1.createOrReplaceTempView("temp")

fraud_merchants.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


INNER JOIN temp2

ON temp.merchant_abn = temp2.merchant_abn_1
""")

In [7]:
#Changing all null probability values to 0.01: 

MIN_P = 0.01
final_join = final_join.fillna(MIN_P, subset=["cons_fraud_prob", "merch_fraud_prob"])

In [8]:
#join segment data 
#clustering 
#decide threshold based on clusters 
#use threshold to classify if transaction is fradulent or not 

In [9]:
# Reading in segments 
segments_data = spark.read.csv('/Users/ahirve/Desktop/ads_bnpl/data/curated/tagged_merchants.csv')

In [10]:
# segments_data.limit(5)

In [11]:
#Removing false header line 
segments_data.na.drop(subset=["_c0"])

_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,Felis Limited,"((furniture, home...",10023283211,furniture home fu...,0,Furniture
1,Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,cable satellite a...,4,Electronics
2,Nunc Sed Company,"([jewelry, watch,...",10165489824,jewelry watch clo...,1,Toys and DIY
3,Ultricies Digniss...,"([wAtch, clock, a...",10187291046,watch clock and j...,1,Toys and DIY
4,Enim Condimentum PC,([music shops - m...,10192359162,music shops musi...,3,"Books, Stationary..."
5,Fusce Company,"[(gift, card, nov...",10206519221,gift card novelty...,4,Electronics
6,Aliquam Enim Inco...,"[(computers, comP...",10255988167,computers compute...,4,Electronics
7,Ipsum Primis Ltd,"[[watch, clock, a...",10264435225,watch clock and j...,1,Toys and DIY
8,Pede Ultrices Ind...,([computer progra...,10279061213,computer programm...,2,"Beauty, Health, P..."
9,Nunc Inc.,"[(furniture, home...",10323485998,furniture home fu...,0,Furniture


In [12]:
final_join.createOrReplaceTempView("temp")

segments_data.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2._c3
""")

In [13]:
# final_join.limit(5)

In [14]:
#Creating a table of average fraud probabilities for each category of purchase: 

aggregated_prob = final_join.groupBy("temp2._c6").agg(avg("cons_fraud_prob").alias("avg_consumer_fraud"),avg("merch_fraud_prob").alias("avg_merchant_fraud"))

In [15]:
aggregated_prob.show()

+--------------------+------------------+------------------+
|                 _c6|avg_consumer_fraud|avg_merchant_fraud|
+--------------------+------------------+------------------+
|         Electronics|15.085244801372209| 29.63518320125268|
|        Toys and DIY|15.842853538478918| 32.40489326031825|
|           Furniture|15.077896208142434|30.941479265167136|
|Beauty, Health, P...|15.392330299269045| 29.76192327324857|
|Books, Stationary...|15.140863705245456|29.016133114054462|
+--------------------+------------------+------------------+



In [17]:
aggregated_prob.limit(6)

_c6,avg_consumer_fraud,avg_merchant_fraud
Electronics,15.085244801372209,29.63518320125268
Toys and DIY,15.842853538478918,32.40489326031825
Furniture,15.077896208142434,30.941479265167136
"Beauty, Health, P...",15.392330299269045,29.76192327324857
"Books, Stationary...",15.140863705245456,29.016133114054462


In [18]:
final_join.limit(5)

merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id,user_id_1,order_datetime_1,cons_fraud_prob,merchant_abn_1,order_datetime,merch_fraud_prob,_c0,_c1,_c2,_c3,_c4,_c5,_c6
21439773999,"cable, satellite,...",a,110.0283925711613,18569,1417071,18569,2021-12-22,9.420406825263207,21439773999,2021-11-26,28.504479048104585,519,Mauris Non Institute,"([cable, satellit...",21439773999,cable satellite a...,4,Electronics
96680767841,motor vehicle sup...,a,244.19185320012286,20924,1051408,20924,2022-02-17,9.76740823700797,96680767841,2021-11-26,29.555244690425017,3881,Ornare Limited,([motor vehicle s...,96680767841,motor vehicle sup...,0,Furniture
18158387243,health and beauty...,c,860.3282463138394,15655,375574,15655,2022-02-10,11.94169979516484,18158387243,2021-11-29,28.956947892226466,378,Nec Tellus Ltd,[[health and beau...,18158387243,health and beauty...,2,"Beauty, Health, P..."
18158387243,health and beauty...,c,860.3282463138394,15655,375574,15655,2022-02-10,11.94169979516484,18158387243,2021-11-26,29.102418916409913,378,Nec Tellus Ltd,[[health and beau...,18158387243,health and beauty...,2,"Beauty, Health, P..."
18158387243,health and beauty...,c,860.3282463138394,15655,375574,15655,2022-01-14,13.321204535044247,18158387243,2021-11-29,28.956947892226466,378,Nec Tellus Ltd,[[health and beau...,18158387243,health and beauty...,2,"Beauty, Health, P..."


In [24]:
#adding a column for is_fraud for each transaction: 

is_fraud = when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Toys and DIY") & (final_join["cons_fraud_prob"] > 15.843) &(final_join["merch_fraud_prob"]>32.404),1)\
.when((final_join["_c6"] == "Furniture") & (final_join["cons_fraud_prob"] > 15.077) & (final_join["merch_fraud_prob"]>30.941),1)\
.when((final_join["_c6"] == "Beauty, Health, Personal and Household") & (final_join["cons_fraud_prob"] > 15.392) & (final_join["merch_fraud_prob"]>29.76),1)\
.when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Books, Stationary and Music") & (final_join["cons_fraud_prob"] > 15.140) &(final_join["merch_fraud_prob"]>29.016),1).otherwise(0)



In [25]:
model_with_fraud = final_join.withColumn("is_fraud",is_fraud)

In [26]:
model_with_fraud.limit(5)

merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id,user_id_1,order_datetime_1,cons_fraud_prob,merchant_abn_1,order_datetime,merch_fraud_prob,_c0,_c1,_c2,_c3,_c4,_c5,_c6,is_fraud
21439773999,"cable, satellite,...",a,110.0283925711613,18569,1417071,18569,2021-12-22,9.420406825263207,21439773999,2021-11-26,28.504479048104585,519,Mauris Non Institute,"([cable, satellit...",21439773999,cable satellite a...,4,Electronics,0
96680767841,motor vehicle sup...,a,244.19185320012286,20924,1051408,20924,2022-02-17,9.76740823700797,96680767841,2021-11-26,29.555244690425017,3881,Ornare Limited,([motor vehicle s...,96680767841,motor vehicle sup...,0,Furniture,0
18158387243,health and beauty...,c,860.3282463138394,15655,375574,15655,2022-02-10,11.94169979516484,18158387243,2021-11-29,28.956947892226466,378,Nec Tellus Ltd,[[health and beau...,18158387243,health and beauty...,2,"Beauty, Health, P...",0
18158387243,health and beauty...,c,860.3282463138394,15655,375574,15655,2022-02-10,11.94169979516484,18158387243,2021-11-26,29.102418916409913,378,Nec Tellus Ltd,[[health and beau...,18158387243,health and beauty...,2,"Beauty, Health, P...",0
18158387243,health and beauty...,c,860.3282463138394,15655,375574,15655,2022-01-14,13.321204535044247,18158387243,2021-11-29,28.956947892226466,378,Nec Tellus Ltd,[[health and beau...,18158387243,health and beauty...,2,"Beauty, Health, P...",0


In [ ]:
model_with_fraud.write.parquet("/Users/ahirve/Desktop/ads_bnpl/data/curated/model_data_with_fraud_col.") 